In [ ]:
import json
import mysql.connector
import uuid


def get_db_cursor():
    db = mysql.connector.connect(host="localhost", user="root", password="ayanali78941", database="warehousing_project")
    return db, db.cursor()


def get_uuid_string():
    return str(uuid.uuid4())


def add_main_categories(db, cursor):
    file_object = open("C:\\Users\\themg\\PycharmProjects\\Projects-For-Portfolio\\daraz-scraep\\input-links-products.json")
    file_pointer = json.load(file_object)

    sql_main_categories = "INSERT INTO daraz_main_categories (id, main_category_name) VALUES (%s, %s)"
    sql_sub_categories = "INSERT INTO daraz_sub_categories (id, sub_category_name) VALUES (%s, %s)"
    for i in range(0, len(file_pointer)):
        uuid_id_main_cat = get_uuid_string()
        value = list(file_pointer[i].keys())[0]
        values_main_cat = (uuid_id_main_cat, value)
        sub_categories_list = file_pointer[i][value]

        for single_sub_cat in sub_categories_list:
            uuid_id_sub_cat = get_uuid_string()

            values_sub_cat = (uuid_id_sub_cat, single_sub_cat)
            cursor.execute(sql_sub_categories, values_sub_cat)
        cursor.execute(sql_main_categories, values_main_cat)
    db.commit()
    return


def link_main_sub_cat(db, cursor):
    file_object = open("C:\\Users\\themg\\PycharmProjects\\Projects-For-Portfolio\\daraz-scraep\\input-links-products.json")
    file_pointer = json.load(file_object)

    sql_main_cat = "SELECT id FROM daraz_main_categories WHERE main_category_name = %s"
    sql_sub_cat = "SELECT id FROM daraz_sub_categories WHERE sub_category_name = %s"
    sql_insert_relation = "INSERT INTO main_sub_categories_relation (main_category_id, sub_category_id) VALUES (%s, %s)"

    for i in range(0, len(file_pointer)):
        main_cat_key = list(file_pointer[i].keys())[0]
        sub_cat_list = file_pointer[i][main_cat_key]

        cursor.execute(sql_main_cat, (main_cat_key, ))
        main_cat_id_result = cursor.fetchall()[0][0]
        for single_cat_name in sub_cat_list:
            cursor.execute(sql_sub_cat, (single_cat_name, ))
            sub_cat_id_result = cursor.fetchall()[0][0]
            cursor.execute(sql_insert_relation, (main_cat_id_result, sub_cat_id_result))
    db.commit()


def add_links_to_db(db, cursor):
    file_object = open("C:\\Users\\themg\\PycharmProjects\\Projects-For-Portfolio\\daraz-scraep\\final-output-file.json")
    file_pointer = json.load(file_object)

    sql_sub_cat_id = "SELECT id FROM daraz_sub_categories WHERE sub_category_name = %s"
    sql_products = "INSERT INTO products (product_link, sub_category_id) VALUES (%s, %s)"

    try:
        for i in range(0, len(file_pointer)):
            main_cat_key = list(file_pointer[i].keys())[0]
            sub_cat_list = file_pointer[i][main_cat_key]
            for single_sub_cat_object in sub_cat_list:
                sub_cat_name = list(single_sub_cat_object.keys())[0]
                cursor.execute(sql_sub_cat_id, (sub_cat_name, ))
                sub_cat_id_result = cursor.fetchall()[0][0]
                for single_link in single_sub_cat_object[sub_cat_name]:
                    cursor.execute(sql_products, (single_link, sub_cat_id_result))
            print(f'{i}-done')
    except Exception:
        db.commit()
        return
    db.commit()



def main_caller():
    db_cursor = get_db_cursor()

    add_main_categories(*db_cursor)
    link_main_sub_cat(*db_cursor)
    add_links_to_db(*db_cursor)

    db_cursor[0].commit()
    db_cursor[1].close()
    db_cursor[0].close()


main_caller()

